<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-базы-данных" data-toc-modified-id="Обзор-базы-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор базы данных</a></span></li><li><span><a href="#Выполнение-запросов-к-базе-данных" data-toc-modified-id="Выполнение-запросов-к-базе-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Выполнение запросов к базе данных</a></span><ul class="toc-item"><li><span><a href="#Подсчёт-количества-книг,-вышедших-после-1-января-2000-года" data-toc-modified-id="Подсчёт-количества-книг,-вышедших-после-1-января-2000-года-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Подсчёт количества книг, вышедших после 1 января 2000 года</a></span></li><li><span><a href="#Подсчёт-количества-обзоров-и-средней-оценки-для-каждой-книги" data-toc-modified-id="Подсчёт-количества-обзоров-и-средней-оценки-для-каждой-книги-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Подсчёт количества обзоров и средней оценки для каждой книги</a></span></li><li><span><a href="#Определения-издательства,-которое-выпустило-наибольшее-число-книг-толще-50-страниц" data-toc-modified-id="Определения-издательства,-которое-выпустило-наибольшее-число-книг-толще-50-страниц-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Определения издательства, которое выпустило наибольшее число книг толще 50 страниц</a></span></li><li><span><a href="#Определение-автора-с-самой-высокой-средней-оценкой-книг-(книги-с-50-и-более-оценками)" data-toc-modified-id="Определение-автора-с-самой-высокой-средней-оценкой-книг-(книги-с-50-и-более-оценками)-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Определение автора с самой высокой средней оценкой книг (книги с 50 и более оценками)</a></span></li><li><span><a href="#Расчёт-среднего-количества-обзоров-от-пользователей,-которые-поставили-больше-50-оценок" data-toc-modified-id="Расчёт-среднего-количества-обзоров-от-пользователей,-которые-поставили-больше-50-оценок-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Расчёт среднего количества обзоров от пользователей, которые поставили больше 50 оценок</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Изучение приложения для чтения

Компания купила крупный сервис для чтения книг по подписке. Задача аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Цель работы:** проанализировать базу данных сервиса для чтения книг по подписке.
    
**Задачи работы:**

1. Посчитать, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитать количество обзоров и среднюю оценку;
3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц (исключить брошюры);
4. Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;
5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

**Исходные данные:** база данных с информацией о книгах, издательствах, авторах, а также пользовательские обзоры книг.

**Описание данных:**
    
Таблица `books` содержит данные о книгах:
    
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Таблица `authors` содержит данные об авторах:
    
- `author_id `— идентификатор автора;
- `author` — имя автора.

Таблица `publishers` содержит данные об издательствах:
    
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства.

Таблица `ratings` содержит данные о пользовательских оценках книг:
    
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица `reviews` содержит данные о пользовательских обзорах:
    
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.

*Импортируем необходимые для работы библиотеки.*

In [1]:
import pandas as pd
from sqlalchemy import create_engine

*Подготовим подключение к базе данных.*

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

*Объявим функцию для выполнения запросов к БД.*

In [3]:
def sql_query(query):
    return pd.io.sql.read_sql(query, 
                              con = engine)

## Обзор базы данных

*Создадим запрос для вывода первых 5 строк таблицы `books` на экран.*

In [4]:
query = '''
           SELECT * 
           FROM books
           LIMIT 5
        '''

*Выведем первые 5 строк таблицы `books` на экран.*

In [5]:
sql_query(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


*Создадим запрос для вывода первых 5 строк таблицы `authors` на экран.*

In [6]:
query = '''
           SELECT * 
           FROM authors
           LIMIT 5
        '''

*Выведем первые 5 строк таблицы `authors` на экран.*

In [7]:
sql_query(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


*Создадим запрос для вывода первых 5 строк таблицы `publishers` на экран.*

In [8]:
query = '''
           SELECT * 
           FROM publishers
           LIMIT 5
        '''

*Выведем первые 5 строк таблицы `publishers` на экран.*

In [9]:
sql_query(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


*Создадим запрос для вывода первых 5 строк таблицы `ratings` на экран.*

In [10]:
query = '''
           SELECT * 
           FROM ratings
           LIMIT 5
        '''

*Выведем первые 5 строк таблицы `ratings` на экран.*

In [11]:
sql_query(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


*Создадим запрос для вывода первых 5 строк таблицы `reviews` на экран.*

In [12]:
query = '''
           SELECT * 
           FROM reviews
           LIMIT 5
        '''

*Выведем первые 5 строк таблицы `reviews` на экран.*

In [13]:
sql_query(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


##  Выполнение запросов к базе данных

### Подсчёт количества книг, вышедших после 1 января 2000 года

*Создадим запрос для расчёта количества книг, вышедших после 1 января 2000 года.*

In [14]:
query = '''
           SELECT COUNT(book_id) 
           FROM books
           WHERE publication_date > '2000-01-01';
        '''

*Посчитаем количество книг, вышедших после 1 января 2000 года.*

In [15]:
sql_query(query)

,count
0,819


**После 1 января 2000 года было выпущено 819 книг. Маловато, однако.**

### Подсчёт количества обзоров и средней оценки для каждой книги

*Создадим запрос для подсчёта количества обзоров и средней оценки для каждой книги.*

In [16]:
query = '''
           SELECT b.book_id,
                  b.title,
                  COUNT(DISTINCT(rw.review_id)) AS review_count,
                  AVG(rt.rating) AS avg_rating
           FROM books AS b
           LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
           LEFT JOIN reviews AS rw ON b.book_id = rw.book_id
           GROUP BY b.book_id
           ORDER BY COUNT(DISTINCT(rw.review_id)) DESC, AVG(rt.rating) DESC
           
        '''

*Выведем вычисленные количество обзоров и средние оценки для каждой книги.*

In [17]:
sql_query(query)

,book_id,title,review_count,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,656,The Book Thief,6,4.264151
4,734,The Glass Castle,6,4.206897
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
996,387,Leonardo's Notebooks,0,4.000000
997,221,Essential Tales and Poems,0,4.000000
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


**Больше всего обзоров (7) у книги "Сумерки", рейтинг при этом не слишком высокий (3.7). По 6 обзоров и наиболее высокие оценки у книг "Гарри Поттер и узник Азкабана" (4.4), "Гарри Поттер и тайная комната" (4.3), "Книжный вор" (4.3) и "Замок из стекла" (4.2).**

### Определения издательства, которое выпустило наибольшее число книг толще 50 страниц

*Создадим запрос для определения издательства, которое выпустило наибольшее число книг толще 50 страниц.*

In [18]:
query = '''
           WITH 
           pub AS (SELECT COUNT(b.book_id) AS thin_books_count,
                          p.publisher
                   FROM books AS b
                   LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
                   WHERE b.num_pages > 50
                   GROUP BY p.publisher)
                      
           SELECT publisher
           FROM pub
           WHERE thin_books_count = (SELECT MAX(thin_books_count)
                                     FROM pub)
                      
        '''

*Выведем название издательства, которое выпустило наибольшее число книг толще 50 страниц.*

In [19]:
sql_query(query)

,publisher
0,Penguin Books


**Больше всего книг выпустило издательство "Penguin Books".**

### Определение автора с самой высокой средней оценкой книг (книги с 50 и более оценками)

*Создадим запрос для определения автора с самой высокой средней оценкой книг (учтём книги с 50 и более оценками).*

In [20]:
query = '''
           WITH 
           au AS (SELECT a.author,
                         AVG(r.rating) AS avg_rating
                  FROM authors AS a
                  LEFT JOIN books AS b ON a.author_id = b.author_id
                  LEFT JOIN ratings AS r ON b.book_id = r.book_id
                  WHERE b.book_id IN (SELECT b.book_id
                                      FROM books AS b
                                      LEFT JOIN ratings AS r ON b.book_id = r.book_id
                                      GROUP BY b.book_id
                                      HAVING COUNT(r.rating) >= 50)
                  GROUP BY a.author_id)
                      
            SELECT author,
                   avg_rating
            FROM au
            WHERE avg_rating = (SELECT MAX(avg_rating)
                                FROM au)
                      
        '''

*Выведем автора с самой высокой средней оценкой книг.*

In [21]:
sql_query(query)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


**Джоан Роулинг, указанная в соавторстве с иллюстратором "Гарри Поттера", занимает первое место в рейтинге писателей с самой высокой средней оценкой книг.**

### Расчёт среднего количества обзоров от пользователей, которые поставили больше 50 оценок

*Создадим запрос для расчёта среднего количества обзоров от пользователей, которые поставили больше 50 оценок.*

In [22]:
query = '''
           WITH 
           u AS (SELECT username,
                 COUNT(rating_id)
                 FROM ratings
                 GROUP BY username
                 HAVING COUNT(rating_id) > 50
                 ORDER BY COUNT(rating_id) DESC)
                      
           SELECT AVG(review_count)
           FROM (SELECT u.username,
                        COUNT(r.review_id) AS review_count
                 FROM u
                 LEFT JOIN reviews AS r ON u.username = r.username
                 GROUP BY u.username) AS r
                      
                      
         '''

*Выведем результат расчёта среднего количества обзоров от пользователей, которые поставили больше 50 оценок.*

In [23]:
sql_query(query)

,avg
0,24.333333


**Пользователи, которые поставили больше 50 оценок, в среднем написали 24 обзора.**

## Выводы

 **В ходе исследования была проанализирована база данных сервиса для чтения книг по подписке. В результате исследования сделаны следующие выводы:**
 
1. После 1 января 2000 года было выпущено 819 книг.
2. Больше всего обзоров (7) у книги "Сумерки", рейтинг при этом не слишком высокий (3.7). По 6 обзоров и наиболее высокие оценки у книг "Гарри Поттер и узник Азкабана" (4.4), "Гарри Поттер и тайная комната" (4.3), "Книжный вор" (4.3) и "Замок из стекла" (4.2).
3. Больше всего книг выпустило издательство "Penguin Books".
4. Джоан Роулинг занимает первое место в рейтинге писателей с самой высокой средней оценкой книг (4.3).
5. Пользователи, которые поставили больше 50 оценок, в среднем написали 24 обзора.
 
**На основании проведённого исследования можно заключить, что пользователи приложения преимущественно современные молодые люди, читающие книги разных жанров.**